In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from natsort import natsorted
import numpy as np
import matplotlib.animation as animation
import xarray as xr
import imageio
import matplotlib
import sys
import os
from tidy3d import material_library

# Assuming /AutomationModule is in the root directory of your project
sys.path.append(os.path.abspath(rf'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM

In [2]:
tidy3dAPI = os.environ["API_TIDY3D_KEY"]


In [3]:
# medium = material_library['Au'].medium
medium = material_library['Au']['Olmon2012crystal']
run = True
lambdas = np.array([0.54,0.52])

In [4]:
folder_path=rf"H:\phd stuff\tidy3d\structures\2D SHU"
project_name = fr"20250710 Cubic Slab Gold_Olmon2012crystal Nanoparticles"
runtime_ps = 20e-12
min_steps_per_lambda = 20

In [5]:
structure_1 = AM.loadAndRunStructure(key = tidy3dAPI
                    ,direction="z", lambda_range=lambdas,
                    box_size= 1.22,runtime_ps=runtime_ps,min_steps_per_lambda=min_steps_per_lambda,
                   scaling=1,shuoff_condtion=1e-20, verbose=True, boundaries="periodic", absorbers=200,
                   monitors=["flux"],cell_size_manual=4,flux_monitor_position=1.1,
                   freqs=500,  source="planewave", gaussian_params={"waist_radius":1,"waist_distance":2,"theta":0,"phi":0, "size":0.8},
                   width=0.4, ref_only=True
                   )

Configured successfully.


In [6]:
def random_points_in_sphere(n_points, radius=6.0):
    # Generate random directions (normal distribution, then normalize)
    vec = np.random.normal(size=(n_points, 3))
    vec /= np.linalg.norm(vec, axis=1)[:, np.newaxis]

    # Generate radii with cubic root to ensure uniform distribution in volume
    r = radius * np.random.uniform(0, 1, size=n_points) ** (1/3)

    # Scale unit vectors by random radii
    points = vec * r[:, np.newaxis]
    return points

In [7]:
n = 1500  # particles
sphere_r = 0.6
# positions = random_points_in_sphere(1500, radius=sphere_r)
positions =  np.random.uniform(-sphere_r, sphere_r, size=(n, 3))

In [8]:
particle_group = []
for x,y,z in positions:
    particle_group.append(td.Sphere(center=[x, y, z], radius=0.02))

particles = td.Structure(geometry=td.GeometryGroup(geometries=particle_group), medium=medium)


In [9]:
lambda_0 = 0.532
freq_0 = td.C_0/lambda_0
source_def = td.PlaneWave(
        source_time = td.GaussianPulse(
            freq0=td.C_0/lambda_0,
            fwidth=np.array([freq_0*0.1])
        ),
        size= (0 if structure_1.direction == "x" else  td.inf, 
              0 if structure_1.direction == "y" else td.inf, 
              0 if structure_1.direction == "z" else  td.inf
              ) 
              ,
        center=((-structure_1.Lx*0.5+structure_1.spacing*0.1) if structure_1.direction == "x" else structure_1.gaussian_params.get("position_x", 0), 
                (-structure_1.Ly*0.5+structure_1.spacing*0.1) if structure_1.direction == "y" else  structure_1.gaussian_params.get("position_y", 0), 
                -1.5),

        direction='+',
        # pol_angle=structure_1.pol_angle,
        # angle_phi=  structure_1.gaussian_params.get("phi", 0),
        # angle_theta= structure_1.gaussian_params.get("theta", 0),
        # waist_distance=structure_1.gaussian_params["waist_distance"],
        # waist_radius=structure_1.gaussian_params["waist_radius"],
        name='planewave',
        )

monitor_1 = td.FluxMonitor(
                center = (
                            0 if structure_1.flux_monitor_position else ((structure_1.Lx - structure_1.spacing)*0.5 if structure_1.direction == "x" else 0), 
                            0 if structure_1.flux_monitor_position else ((structure_1.Ly - structure_1.spacing)*0.5 if structure_1.direction == "y" else 0), 
                            structure_1.flux_monitor_position if structure_1.flux_monitor_position else ((structure_1.Lz - structure_1.spacing)*0.5 if structure_1.direction == "z" else 0)
                            ),
                size = (
                    0 if structure_1.direction == "x" else td.inf, 
                    0 if structure_1.direction == "y" else td.inf, 
                    0 if structure_1.direction == "z" else td.inf, 
                    ),
                freqs = np.linspace(freq_0 - 2 * freq_0*0.1, freq_0 + 2 * freq_0*0.1, 25),
                name='flux1' #To the right 
            )
monitor_2 = td.FluxMonitor(
                center = (
                        (-structure_1.Lx+structure_1.spacing)*0.5 if structure_1.direction =="x" else 0, 
                        (-structure_1.Ly+structure_1.spacing)*0.5 if structure_1.direction =="y" else 0, 
                        -structure_1.flux_monitor_position if structure_1.flux_monitor_position else ((-structure_1.Lz+structure_1.spacing)*0.5 if structure_1.direction =="z" else 0)
                        ),
                size = (
                    0 if structure_1.direction == "x" else td.inf, 
                    0 if structure_1.direction == "y" else td.inf, 
                    0 if structure_1.direction == "z" else td.inf
                    ),
                freqs =np.linspace(freq_0 - 2 * freq_0*0.1, freq_0 + 2 * freq_0*0.1, 25),
                name='flux2'#To the left
            )

In [10]:
sim = structure_1.sim.copy(update={'structures':[particles],"sources":[source_def],"monitors":[monitor_1,monitor_2]})
sim_name=f"Au_N_{n}_r_{sphere_r*1e3}_nm"

In [11]:
sim.plot_3d()

In [12]:
if run:
    file_path = rf"H:\phd stuff\tidy3d\data/{project_name}"

    if os.path.exists(os.path.join(file_path,f"{sim_name}.txt")):
        print("Exist!!")
    else:
        id0 =web.upload(sim.copy(update={'structures':[]}), folder_name=project_name,task_name=fr"{sim_name}_ref", verbose=True)
        web.start(task_id = id0)
        web.monitor(id0)

        id =web.upload(sim, folder_name=project_name,task_name=sim_name, verbose=True)
        ids = id0+ '\n' + id
        # Check if the folder exists
        os.makedirs(file_path,exist_ok=True)

        # Open file in write mode
        with open(os.path.join(file_path,f"{sim_name}.txt"), "w") as f:
            # Write the string to the file
            f.write(ids)

        web.start(task_id = id)
        web.monitor(id)

                
else:
    raise Exception
    id =web.upload(sim,task_name=sim_name, verbose=True)
    print(web.estimate_cost(id))


17:12:10 W. Europe Daylight Time Created task 'Au_N_1500_r_600.0_nm_ref' with   
                                 task_id                                        
                                 'fdve-061cdc7e-756d-4f3a-99ec-12791b793a8c' and
                                 task_type 'FDTD'.

                                 View task using web UI at                      
                                 ]8;id=149065;https://tidy3d.simulation.cloud/workbench?taskId=fdve-061cdc7e-756d-4f3a-99ec-12791b793a8c\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=115116;https://tidy3d.simulation.cloud/workbench?taskId=fdve-061cdc7e-756d-4f3a-99ec-12791b793a8c\task]8;;\
                                 ]8;id=115116;https://tidy3d.simulation.cloud/workbench?taskId=fdve-061cdc7e-756d-4f3a-99ec-12791b793a8c\Id]8;;\]8;id=149065;https://tidy3d.simulation.cloud/workbench?taskId=fdve-061cdc7e-756d-4f3a-99ec-12791b793a8c\=]8;;\]8;id=754581;https://tidy3d.simulation.cloud/workbench?taskId=fdve-061cdc7e-756d-4f3a-99ec-12791b793a8c\fdve]8;;\]8;id=149065;https://tidy3d.simulation.cloud/workbench?taskId=fdve-061cdc7e-756d-4f3a-99ec-12791b793a8c\-061cdc7e-756d-4f3a-99ec-12791b793a8c']8;;\.

                                 Task folder: ]8;id=805628;https://tidy3d.simulation.cloud/folders/folder-711aa94a-906c-4c0e-a0b9-a79727fc5c38\'20250710 Cubic Slab ]8;;\             
                                 ]8;id=805628;https://tidy3d.simulation.cloud/folders/folder-711aa94a-906c-4c0e-a0b9-a79727fc5c38\Gold_Olmon2012crystal Nanoparticles']8;;\.

Output()

17:12:12 W. Europe Daylight Time Maximum FlexCredit cost: 0.149. Minimum cost   
                                 depends on task execution details. Use         
                                 'web.real_cost(task_id)' to get the billed     
                                 FlexCredit cost after a simulation run.

...

Output()

17:17:17 W. Europe Daylight Time status = postprocess

Output()

17:17:20 W. Europe Daylight Time status = success

17:17:22 W. Europe Daylight Time View simulation result at                      
                                 ]8;id=293862;https://tidy3d.simulation.cloud/workbench?taskId=fdve-159d8b39-d313-48a9-bf95-b62f3ca0d93a\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=353983;https://tidy3d.simulation.cloud/workbench?taskId=fdve-159d8b39-d313-48a9-bf95-b62f3ca0d93a\task]8;;\
                                 ]8;id=353983;https://tidy3d.simulation.cloud/workbench?taskId=fdve-159d8b39-d313-48a9-bf95-b62f3ca0d93a\Id]8;;\]8;id=293862;https://tidy3d.simulation.cloud/workbench?taskId=fdve-159d8b39-d313-48a9-bf95-b62f3ca0d93a\=]8;;\]8;id=915458;https://tidy3d.simulation.cloud/workbench?taskId=fdve-159d8b39-d313-48a9-bf95-b62f3ca0d93a\fdve]8;;\]8;id=293862;https://tidy3d.simulation.cloud/workbench?taskId=fdve-159d8b39-d313-48a9-bf95-b62f3ca0d93a\-159d8b39-d313-48a9-bf95-b62f3ca0d93a']8;;\.

In [13]:
1.1**5

1.6105100000000006